In [3]:
import os

import gensim
import pyLDAvis.gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

# Folder path
folder = './run_data/'
folder_traing = './training/'
dest_file = './final/final_content.csv'

%matplotlib inline

In [5]:
entries = os.listdir(folder_traing)
print("Total files: ", len(entries))

# Print out file names
print([entry for entry in entries[:5]])

Total files:  23297
['100634927493693440.txt', '100691512974778368.txt', '100997287899172864.txt', '101321977507229696.txt', '101340793482125312.txt']


In [6]:
# Read file and push to list\n",
list_text = []
# list_text.append([f.read() with open(file,'r') as f for file in entries"]

for entry in entries:
    file = folder_traing + entry
    with open(file, 'r') as f:
        data = f.read()
#         print("Read: ", len(data))
        list_text.append(data)
#         print("====================\n")

In [7]:
# Split data into training & testing
X_train = list_text[:11000]
X_test = list_text[11001::]

## 1. Define Function

In [21]:
m_stopwords = gensim.parsing.preprocessing.STOPWORDS
punc = ['ill','kill']
m_stopwords.union(punc)
type(m_stopwords)

frozenset

In [22]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        # Give word . Return the fequency of word\n",
        # Exclude STOPWORD, everything very less then 3\n",
        if token not in m_stopwords  and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [23]:
print(preprocess(X_train[100]))

['reidst', 'john', 'senior', 'fit', 'editor', 'write', 'frequent', 'product', 'review', 'healthcom', 'lisa', 'halbowerfenton', 'footwear', 'product', 'develop', 'consult', 'special', 'analysi', 'structur', 'jessica', 'matthew', 'person', 'trainer', 'plus', 'certif', 'director', 'gear', 'review', 'american', 'council', 'exercis', 'kristin', 'mcgee', 'yoga', 'pilat', 'instructor', 'person', 'trainer', 'regular', 'columnist', 'natali', 'digat', 'muth', 'physician', 'regist', 'dietician', 'person', 'trainer', 'spokesperson', 'american', 'council', 'exercis']


In [24]:
print(gensim.utils.simple_preprocess(X_train[100]))

['su', 'reidst', 'john', 'is', 'senior', 'fitness', 'editor', 'she', 'writes', 'frequent', 'product', 'reviews', 'for', 'healthcom', 'lisa', 'halbowerfenton', 'is', 'footwear', 'product', 'development', 'consultant', 'specializing', 'in', 'analysis', 'and', 'structure', 'jessica', 'matthews', 'is', 'personal', 'trainer', 'plus', 'the', 'certification', 'director', 'and', 'gear', 'reviewer', 'for', 'the', 'american', 'council', 'on', 'exercise', 'kristin', 'mcgee', 'is', 'yoga', 'and', 'pilates', 'instructor', 'and', 'personal', 'trainer', 'as', 'well', 'as', 'regular', 'columnist', 'natalie', 'digate', 'muth', 'md', 'mph', 'rd', 'is', 'physician', 'and', 'registered', 'dietician', 'as', 'well', 'as', 'personal', 'trainer', 'and', 'spokesperson', 'for', 'the', 'american', 'council', 'on', 'exercise']


In [25]:
processed_docs = []

for doc in X_train:
    processed_docs.append(preprocess(doc))

## 2. PreProcessing

In [26]:
# Push to dictionary
dictionary = gensim.corpora.Dictionary(processed_docs)

In [27]:
# "# I dont want words no_below, no_above
dictionary.filter_extremes(no_below=10, no_above=0.2,keep_n=1000)

In [28]:
# "# Bag of words
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [29]:
# Fit LDA model\n",
# num_topic [dont know] sothat pick a proper number,
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=8,
                                      id2word=dictionary,
                                     passes=10, workers=2)

In [30]:
# LDA help vizualize
lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_display)

C:\Users\Shaynie\Anaconda3\envs\capstone_app\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
